# Python script to download latest DOME Registry contents and maintain up to date file of all entries. (20241202)
## Script will also offer a download of all related EPMC articles for NLP usage through their API

In [14]:
# 1. Use the DOME API to download all entries of the DOME Registry and store in a file 
import requests
from datetime import datetime

# Define the URL for the API call: check the API documentation for the correct URL on the DOME Registry website
url = "https://registry.dome-ml.org/api/review?skip=0&limit=250&text=%20&public=true&sort=publication.year&asc=true"

# Make an API request to the URL
response = requests.get(url, headers={'accept': '*/*'})

# Check if the request was successful
if response.status_code == 200:
    # Get the current date in ISO format for file naming
    current_date = datetime.now().strftime('%Y-%m-%d')
    
    # Create the output file name
    file_name = f"DOME_Registry_Contents_{current_date}.json"
    
    # Save the content to a file
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(response.text)
    
    print(f"Data downloaded and saved to '{file_name}'")
else:
    print(f"Failed to retrieve the data. Status code: {response.status_code}")



Data downloaded and saved to 'DOME_Registry_Contents_2024-12-02.json'


In [49]:
# 2. Produce DOME Registry contents metadata .csv file and data visualisation
import json

# 2.1 Visualise and check the contents of the downloaded json file as print out

# Function to read and pretty-print the JSON file sample entry
def pretty_print_json(file_name):
    try:
        # Open and read the JSON file
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # Pretty-print the JSON data
        print(json.dumps(data, indent=4))
    
    except Exception as e:
        print(f"Error reading the JSON file: {e}")

# Call the function to pretty-print the JSON file
pretty_print_json(file_name)

[
    {
        "_id": "63516fedb9c880af1f305b5c",
        "publication": {
            "pmid": "16524483",
            "updated": "01/28/2022 00:13:56",
            "authors": "Wang H, Zheng H, Simpson D, Azuaje F",
            "journal": "BMC Bioinformatics",
            "title": "Machine learning approaches to supporting the identification of photoreceptor-enriched genes based on expression data.",
            "doi": "10.1186/1471-2105-7-116",
            "year": "2006",
            "done": 0,
            "skip": 0
        },
        "public": true,
        "created": "2022-09-01T15:16:05.444Z",
        "updated": "2022-09-01T15:16:05.444Z",
        "uuid": "66a94333-8cd1-499c-86ef-0497a4c4dabc",
        "shortid": "6i0xepuivt",
        "score": 0.67,
        "matches": {
            "publication/updated": "01/28/2022 00:13:56",
            "publication/authors": "Wang H, Zheng H, Simpson D, Azuaje F",
            "publication/journal": "BMC Bioinformatics",
            "publication

In [ ]:
# 2. Flatten the JSON for easier data processing and write to a new .json file 
import json

# Define the path to the JSON file
file_name = 'DOME_Registry_Contents_2024-12-02.json'  # Replace with your actual file name

# Function to read JSON data
def read_json(file_name):
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"Error reading the JSON file: {e}")
        return None

# Function to flatten JSON
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

# Function to save flattened JSON to a file
def save_flattened_json(flattened_data, output_file_name):
    try:
        with open(output_file_name, 'w', encoding='utf-8') as file:
            json.dump(flattened_data, file, indent=4)
        print(f"Flattened JSON data saved to '{output_file_name}'")
    except Exception as e:
        print(f"Error saving the flattened JSON file: {e}")

# Read JSON data
data = read_json(file_name)

# Flatten JSON data and save to a new file
if data:
    flattened_data = [flatten_json(entry) for entry in data]
    output_file_name = 'flattened_DOME_Registry_Contents.json'
    save_flattened_json(flattened_data, output_file_name)
    
    # Print the flattened JSON data to view it
    print(json.dumps(flattened_data, indent=4))
else:
    print("No data to process.")

Flattened JSON data saved to 'flattened_DOME_Registry_Contents.json'
[
    {
        "_id": "63516fedb9c880af1f305b5c",
        "publication_pmid": "16524483",
        "publication_updated": "01/28/2022 00:13:56",
        "publication_authors": "Wang H, Zheng H, Simpson D, Azuaje F",
        "publication_journal": "BMC Bioinformatics",
        "publication_title": "Machine learning approaches to supporting the identification of photoreceptor-enriched genes based on expression data.",
        "publication_doi": "10.1186/1471-2105-7-116",
        "publication_year": "2006",
        "publication_done": 0,
        "publication_skip": 0,
        "public": true,
        "created": "2022-09-01T15:16:05.444Z",
        "updated": "2022-09-01T15:16:05.444Z",
        "uuid": "66a94333-8cd1-499c-86ef-0497a4c4dabc",
        "shortid": "6i0xepuivt",
        "score": 0.67,
        "matches_publication/updated": "01/28/2022 00:13:56",
        "matches_publication/authors": "Wang H, Zheng H, Simpson D,

In [ ]:
import json
import csv

# 3. Work with flattened JSON data to produce a .csv file for metadata calculation on entries

# Define the path to the flattened JSON file
flattened_file_name = 'flattened_DOME_Registry_Contents.json'  # Replace with your actual file name

# Function to read flattened JSON data
def read_flattened_json(file_name):
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"Error reading the flattened JSON file: {e}")
        return None

# Function to write JSON data to a CSV file
def write_json_to_csv(json_data, csv_file_name):
    try:
        # Determine all possible headers from the entire dataset
        headers = set()
        for entry in json_data:
            headers.update(entry.keys())
        headers = list(headers)
        
        # Write data to CSV file
        with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=headers)
            writer.writeheader()
            for entry in json_data:
                writer.writerow(entry)
        
        print(f"JSON data written to '{csv_file_name}'")
    except Exception as e:
        print(f"Error writing to the CSV file: {e}")

# Read flattened JSON data
flattened_data = read_flattened_json(flattened_file_name)

# Process JSON data into CSV
if flattened_data:
    csv_file_name = 'DOME_Registry_Contents.csv'
    write_json_to_csv(flattened_data, csv_file_name)
else:
    print("No data to process.")



JSON data written to 'DOME_Registry_Contents.csv'


In [56]:
import csv
import re
import os

# Define the path to the CSV file
csv_file_name = 'DOME_Registry_Contents.csv'  # Replace with your actual file name

# Define the EPMC regex pattern for PMIDs
pmid_pattern = re.compile(r'^\d{8}$')

# Define the regex pattern for DOIs
doi_pattern = re.compile(r'^10.\d{4,9}/[-._;()/:A-Z0-9]+$', re.IGNORECASE)

# Function to read CSV data
def read_csv(file_name):
    try:
        with open(file_name, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            data = [row for row in reader]
        return data
    except Exception as e:
        print(f"Error reading the CSV file: {e}")
        return None

# Function to check PMIDs and DOIs and generate metadata
def check_pmids_and_dois_and_generate_metadata(data, csv_file_name):
    pmid_valid = 0
    pmid_invalid = 0
    doi_valid = 0
    doi_invalid = 0
    total_entries = len(data)
    
    for row in data:
        pmid = row.get('publication_pmid', '')
        doi = row.get('publication_doi', '')
        
        if pmid_pattern.match(pmid):
            pmid_valid += 1
        else:
            pmid_invalid += 1
        
        if doi_pattern.match(doi):
            doi_valid += 1
        else:
            doi_invalid += 1
    
    # Print the results
    print(f"{pmid_valid} of {total_entries} PMIDs valid")
    print(f"{pmid_invalid} of {total_entries} PMIDs invalid")
    print(f"{doi_valid} of {total_entries} DOIs valid")
    print(f"{doi_invalid} of {total_entries} DOIs invalid")
    
    # Create metadata CSV file
    metadata_file_name = f"Metadata_{os.path.basename(csv_file_name)}"
    with open(metadata_file_name, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Total Entries', total_entries])
        writer.writerow(['Valid PMIDs', pmid_valid])
        writer.writerow(['Invalid PMIDs', pmid_invalid])
        writer.writerow(['Valid DOIs', doi_valid])
        writer.writerow(['Invalid DOIs', doi_invalid])
    
    print(f"Metadata written to '{metadata_file_name}'")

# Read CSV data
csv_data = read_csv(csv_file_name)

# Check PMIDs and DOIs and generate metadata
if csv_data:
    check_pmids_and_dois_and_generate_metadata(csv_data, csv_file_name)
else:
    print("No data to process.")



211 of 214 PMIDs valid
3 of 214 PMIDs invalid
208 of 214 DOIs valid
6 of 214 DOIs invalid
Metadata written to 'Metadata_DOME_Registry_Contents.csv'


In [ ]:
import csv
import subprocess
import os

# Define the path to the "Valid DOME Registry" CSV file
valid_csv_file_name = 'valid_DOME_Registry_Contents.csv'  # Replace with your actual file name

# Define the output folder for XML files
output_folder = 'EPMC_XML_Files'
os.makedirs(output_folder, exist_ok=True)

# Function to read CSV data
def read_csv(file_name):
    try:
        with open(file_name, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            data = [row for row in reader]
        return data
    except Exception as e:
        print(f"Error reading the CSV file: {e}")
        return None

# Read CSV data
csv_data = read_csv(valid_csv_file_name)

# Retrieve and save XML data for each PMID using bash commands
if csv_data:
    for row in csv_data:
        pmid = row.get('publication_pmid', '')
        if pmid:
            output_file = os.path.join(output_folder, f"{pmid}.xml")
            command = f"curl https://europepmc.org/rest/v1/article/{pmid}/full > {output_file}"
            subprocess.run(command, shell=True)
            print(f"XML data for PMID {pmid} saved to '{output_file}'")
else:
    print("No data to process.")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    81    0    81    0     0    631      0 --:--:-- --:--:-- --:--:--   632
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

XML data for PMID 16524483 saved to 'EPMC_XML_Files/16524483.xml'
XML data for PMID 17374164 saved to 'EPMC_XML_Files/17374164.xml'


100    81    0    81    0     0    588      0 --:--:-- --:--:-- --:--:--   586
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    81    0    81    0     0    639      0 --:--:-- --:--:-- --:--:--   642
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

XML data for PMID 17888165 saved to 'EPMC_XML_Files/17888165.xml'
XML data for PMID 17570862 saved to 'EPMC_XML_Files/17570862.xml'


100    81    0    81    0     0    583      0 --:--:-- --:--:-- --:--:--   586
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    81    0    81    0     0    688      0 --:--:-- --:--:-- --:--:--   692
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

...

XML data for PMID 37882604 saved to 'EPMC_XML_Files/37882604.xml'
XML data for PMID 38000911 saved to 'EPMC_XML_Files/38000911.xml'


100    81    0    81    0     0    633      0 --:--:-- --:--:-- --:--:--   637
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    81    0    81    0     0    551      0 --:--:-- --:--:-- --:--:--   554
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

XML data for PMID 37966428 saved to 'EPMC_XML_Files/37966428.xml'
XML data for PMID 38091508 saved to 'EPMC_XML_Files/38091508.xml'


100    81    0    81    0     0    710      0 --:--:-- --:--:-- --:--:--   710
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    81    0    81    0     0    696      0 --:--:-- --:--:-- --:--:--   698
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

XML data for PMID 38280188 saved to 'EPMC_XML_Files/38280188.xml'
XML data for PMID 39172545 saved to 'EPMC_XML_Files/39172545.xml'
